## Sequência de funções valor para os credores

Queremos resolver por indução retroativa um jogo com T períodos.

Estou trabalhando em como deixar esse jogo em torno do parâmetro T

Teremos duas sequências de funções valor: uma para quando apenas o credor sênior propõe e outra para quando apenas o credor júnior propõe. As funções do jogo final serão uma média ponderada destas.



In [1]:
#para mostrar todos os resultados e não apenas o último
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
#libraries
import numpy as np
from numba import jit, jitclass, float64, njit
import matplotlib.pyplot as plt
%matplotlib inline
import quantecon as qe
from scipy.stats import beta

import math

### Passo 0: definindo comandos do latex para facilitar a escrita

$%conjunto de informações de s em t$
$\newcommand{\Is}[1]{\theta_{s{#1}}, l_{s{#1}}, l_{j{#1}}}$
$%macros para facilitar a escrita de funções valor$


$%conjunto de informações de j em t$
$\newcommand{\Ij}[1]{\theta_{j{#1}}, l_{s{#1}}, l_{j{#1}}}$

$%função valor de s em t$
$\newcommand{\Ws}[1]{ W_{s{#1}} ( \Is{{#1}}) }$

$%função valor de j em t$
$\newcommand{\Wj}[1]{ W_{j{#1}} ( \Ij{{#1}}) }$

$%operador esperança de s em t. 2 argumentos: o primeiro é o período e o segundo é o termo que ela tira a esperança$

$\newcommand{\Es}[2]{\mathbb{E_{#1}^{s} \big[ {#2}  \mid ( \Is{#1} )    \big] }}$

$%minúsculo não mostra o conjunto de informação$
$\newcommand{\es}[2]{\mathbb{E_{#1}^{s} \big[ {#2}  \big] }}$

$%minúsculo não mostra o conjunto de informação$
$%final b de big para aumentar os parênteses$
$\newcommand{\esb}[2]{\mathbb{E_{#1}^{s} \bigg[ {#2}  \bigg] }}$

$%operador esperança de j em t. 2 argumentos: o primeiro é o período e o segundo é o termo que ela tira a esperança$

$\newcommand{\Ej}[2]{\mathbb{E_{#1}^{j} \big[ {#2}  \mid ( \Ij{#1} )    \big] }}$

$%minúsculo não mostra o conjunto de informação$
$\newcommand{\ej}[2]{\mathbb{E_{#1}^{j} \big[ {#2}  \big] }}$

$%minúsculo não mostra o conjunto de informação$
$%final b para aumentar os parênteses$
$\newcommand{\ejb}[2]{\mathbb{E_{#1}^{j} \bigg[ {#2}  \bigg] }}$


$%comando para usar o máximo com chaves grandes$
$\newcommand{\maximo}[1]{\max \bigg\{ #1 \bigg\}}$


In [3]:
#parâmetros do jogo, apenas para ilustração a princípio



μ = 3 #número de meses entre períodos, não entendi onde entra ainda
ρ = 0.9 # (1 - ρ) é a taxa de depreciação da empresa a cada período
β = 1 #usamos aqui a distribuição Uniforme, depois vamos mudar isto
c0 = 0.05 #custo fixo de ir para a corte
c1 = 0.015 #custo variável de ir para a corte




λj = 0.5 #probabilidade de j propor a cada turno. Não precisaremos estimar, isso virá dos dados


θ_s0 = 0.5 #habilidade inicial de s
θ_j0 = 0.5 #habilidade inicial de j

In [4]:
#informações que virão dos dados

Vmax = 100
L = 20 
L_s = 10
L_j = 10

#valores da dívida de cada jogador (virá dos dados, aqui é exemplo):
Ds = 20
Dj = 10

### Qual o ganho que o credor s tem em propor a cada período?

Suponha que ele vai propor reorganizar a firma no perído t. A reorganização demora 1 período, então o valor da firma aparece no próximo período, $V_{t+1}$. E ele precisa fazer uma estimativa sobre $\theta_{s,t+1}$ dada a informação que ele tem no período atual, que eu chamei de k: $\theta_{s,k}$:

$$ R_t (\theta_{s,k}) = V_{t+1} \bigg( \mathbb{E}_k^s \big[ \theta_{s,t+1} \mid \theta_{s,k} \big] \bigg) - L_j$$


#### Qual a estimativa da habilidade dele nos períodos à frente dada a informação dele hoje?

Essa parte depende da hipótese sobre a distribuição da habilidade.

Se supormos que a habilidade segue distribuição Uniforme, a forma fechada dela é:

Suponha $ t > k$

$$ \mathbb{E}_k^s \big[ \theta_{s,t} \mid \theta_{s,k} \big] =  \frac{\theta_{s,k}} {2^{t - k}} + \sum_{n=1}^{t - k} \frac{1}{2^{n}} $$


#### Quais os passos do algoritmo para implementar essa estratégia?

1. Calcular todos os valores de reorganização $ R_t $ de $ t \in  \{k, k+1, ..., T\}$ e também o valor de nunca reorganizar, que é $L_s$

2. Criar uma condição para checar se $\max \{R_{k}, R_{k+1}, ..., R_{T}, L_s \} = R_{k}$
    * em caso positivo, oferecer $P_s = W_{j, t+1} = L_j$ a fim de reorganizar a firma
    * em caso negativo, oferecer $P_s < W_{j, t+1} = L_j$ a fim de adiantar o jogo até o próximo período
    
3. Repetir os passos 1 e 2 até que reorganize a firma ou o jogo termine com ela liquidada

### Passo 1: número máximo de turnos

Calculado com base nos parãmetros

In [5]:
#função para calcular o máximo de turnos do jogo
def maximo_de_turnos(ρ, Vmax, L):
    
    t = 1
    
    while ρ**t * Vmax > L:
        
        t = t + 1
        
    return t
    

In [6]:
#guardando o número máxim de turnos

T = maximo_de_turnos(ρ, Vmax, L)

T

16

In [7]:
#valor máximo de reorganização da firma a cada período


#sequência de valores da firma para cada período

V = np.empty(T, dtype=np.int_)

for t in range(T):

    V[t] = ρ**t * Vmax

In [8]:
#valor de liquidação


#a dívida total é sempre a soma das dívidas
D = Ds + Dj

#o custo total é uma função do tempo

def C(t):
    Ct = c0 * D + c1 * t * D
    
    return Ct

#e os valores de liquidação também são função do tempo

def Ls(t):
    
    Lst = min(L - C(t), Ds)
    
    return Lst

def Lj(t):
    
    Ljt = min(L - C(t) - Ls(t), Dj)
    
    return Ljt
    
    


In [9]:
V[0] #é o primeiro valor do negócio

V[15]

100

20

In [10]:
Ls(2)

17.6

### Passo 2: definir as funções valor de liquidação e as matrizes para guardar seus valores


$$ W_{s1} (\Is{1}) = (1 - \lambda_j) S_1 (\Is{1}) + (\lambda_j) s_1 (\Is{1}) $$

A primeira parte da função valor de s é caso ele venha a propor:


 $$ S_1 (\Is{1}) = \max \bigg\{ L_{s1}, V \es{1}{\theta_s} - \es{1}{W_j}, \es{1}{W_s}
                    \bigg\} $$



In [11]:
#precisamos dos valores de cada theta e cada l para calcular essa função

Ws = np.empty(T, dtype=np.int_)

Wj = np.empty(T, dtype=np.int_)

ls = np.empty(T, dtype=np.int_)

lj = np.empty(T, dtype=np.int_)

θs = np.empty(T, dtype=np.int_)

θj = np.empty(T, dtype=np.int_)


#os valores de continuação no nó final são os valores de liquidação
Ws[T-1] = Ls(T-1)
Wj[T-1] = Lj(T-1)




### Função para tirar os valores esperados de t períodos à frente

Essa é a base do jogo. Vamos começar com a Uniforme mesmo por ser mais fácil de computar (e verificar), depois eu mudo o código para considerar a Beta.

In [12]:
#função para calcular a esperança da habilidade nos períodos à frente
def expec(h, k, t):
    #h é a informação hoje, usada para prever a habilidade
    #k é o período de hoje
    #t é o período para o qual eu desejo estimar a habilidade
    
    #somatório de n até t-k
    summation = 0
    for i in range(1, t-k+1):      
        summation += 2**-i
    
    resultado = h/2**(t-k) + summation
    
    return resultado
    
# expec(0.5, 1, 4) #ok com os meus cálculos

#como ficaria um gráfico dessa função?


In [13]:
expec(θ_s0, 1, (T-1))

0.999969482421875

In [14]:

#aqui a notação está em contagem regressiva novamente
#aqui é o valor do que ele faz
S1 = [Ls((T-1)), V[T-1-1] * expec(θ_s0, 1, (T-1)) - Wj[T-1], Ws[T-1]]


#aqui é o que ele faz. Como policy_S1 é igual a 1, significa que ele propõe reorganizar pagando PS1 = Wj[T-1]
policy_S1 = np.argmax(S1)


S1 = S1[policy_S1]

Es_S1 = S1

A segunda parte da função valor é se se for chamado a responder:
    
$s_1 (\Is{1}) = $
    $ \esb{1}{ Prob ({j liquidar})  \maximo{L_{s1}, V \es{1}{\theta_s} - L_{j1}} } $
    $ + \esb{1}{ Prob ({j reorganizar})   \maximo{\ej{1}{W_s}, \es{1}{W_s}}    }$
 
 
 Sendo que
 
 $ \Es{1}{Prob (j liq em T-1)}$
 
  $$ = Prob \bigg(L_{j1} > \max \{V \es{1}{ \ej{1}{\theta_j} } - \es{1}{ \ej{1}{W_s} }, \es{1}{ \ej{1}{W_j}  }\} \bigg)$$
  
  
 Repare que $L_{j1} > L{j}$ e que $\es{1}{\theta_{j}}$ cresce em $\theta_{j}$. Então deve existir um threshold de reorganização em t = T-1, $\phi_{j1}$, tal que: 

$$V \phi_{j1} - L_{s} = L{j1} $$

$$ \phi_{j1} = \frac{ L_{j1} + L_{s} }{V} $$

Então se $\es{1}{\theta_{j}} > \phi_{j1}$, j reorganiza a firma. E j liquida a firma caso contrário.


A pergunta é: qual a probabilidade de que $\theta_{j}$ tome um valor menor ou igual a $\phi_{j1}$ ? Isso pode ser respondido usando a \textit{prior} do jogador, a função CDF da distribuição Beta:

$$ \Es{1}{Prob (\textnormal{j liq em T-1})} = F_{\beta} \big( \phi_{j1} \mid l_{j1} \big) $$
 


In [15]:
#s1
#Prob_s1 é a probabilidade de l liquidar em t-1, de acordo com o credor sênior (s)


# Prob_s1 

ϕj1 = ( Lj((T-1)) + Ws[T-1] ) / V[T-1]

ϕj1

lj1 = 0.5 #endogeneizar depois


Prob_s1 = 1 - ((1 - ϕj1)**β)/((1 - lj1)**β)

Prob_s1

# a probabilidade de reorganizar será o complementar.

0.55

0.10000000000000009

Agora o valor esperado caso j liquide. No paper isto é 

$  \maximo{L_{s1}, V \es{1}{\theta_s} - L_{j1}}  $

Repare que s quer prever a habilidade dele próprio no período final, então podemos usar a habilidade dele no período T-1 para isso

In [16]:
#vamos criar um vetor de expectativas de s para a sua própria habilidade.

Es_θs = np.ones(T)

#popular esse vetor de expectativas com todos os períodos dali pra frente

θ_s_atual = θ_s0

for i in range(t, T-1):
    
    Es_θs[i-1] = expec(θ_s_atual, t, i)
    

    
Es_θs 

#agora vamos calcular o payoff esperado no caso de j propor liquidação

s1_liq = [Ls((T-1)), V[T-1-1] * Es_θs[T-1] - Lj((T-1)) ]


#aqui é o que ele faz. Como policy_S1 é igual a 1, significa que ele propõe reorganizar pagando PS1 = Wj[T-1]
policy_s1_liq = np.argmax(s1_liq)


s1_liq = s1_liq[policy_s1_liq]


#caso j liquide em T-1, a política ótima é propor reorganizar, o que lhe dá payoff de 22
policy_s1_liq 

s1_liq


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

1

22.0

Agora, o payoff esperado de s caso j decida reorganizar a firma

$ \maximo{\ej{1}{W_s}, \es{1}{W_s}}    $

In [17]:
#agora vamos calcular o payoff esperado no caso de j propor reorganização

s1_reorg = [Ws[T-1], Ws[T-1]]


#aqui é o que ele faz. Como policy_S1 é igual a 1, significa que ele propõe reorganizar pagando PS1 = Wj[T-1]
policy_s1_reorg = np.argmax(s1_reorg)


s1_reorg = s1_reorg[policy_s1_reorg]


#caso j proponha liquidar em T-1, a política ótima é propor reorganizar, o que lhe dá payoff de 22
policy_s1_reorg 

s1_reorg

0

11

A função s1 torna-se então:



In [18]:
#s1 completa

Es_s1 = (Prob_s1) * ( s1_liq) + (1 - Prob_s1) * s1_reorg 




### Como resumir isso?

A função valor no período t é função de lst, ljt e theta_t. Vamos construir uma função que calcula a função valor de cada período para valores arbitrários destas três variáveis.


Passos:

1. Vamos escrever as funções St, st, Jt e jt.

2. Vamos colocar essas funções como argumentos das funções valor

3. Vamos testar as funções valor e ver o que funciona e o que não.


PROBLEMA: as funções ainda estão em função de Wj[t] e Ws[t]



In [19]:
#payoff de S quanto ele tem o direito de propor em t

#aqui eu colo Wj[t] e Ws[t] como argumentos. 
#Tem que ter preparado o terreno antes calculando as funções valor no período seguinte

def St(teta_st, lst, ljt, t):
    
    St = [Ls(t), V[t] * expec(teta_st, t, (t+1)) - Wj[t], Ws[t]]
    
    policy_St = np.argmax(St)
    
    
    
    return (St[policy_St])


    

    


In [20]:
#testando para t = 15, que já sabemos Wj e Ws

St(0.5, 0.5, 0.5, 15)

15.0

In [21]:
#vamos criar os componentes da função st

#probabilidade de j liquidar em t, dado ljt 
def Prob_s(ljt,t):

    #threshold para j tentar liquidar
    ϕjt = ( Lj(t) + Ws[t] ) / V[t]
    
    Prob_st = 1 - ((1 - ϕjt)**β)/((1 - ljt)**β)
    
    #aqui tem que ser no mínimo a igual a zero, apenas por segurança
    
    Prob_st = max(0, Prob_st)
    
    return Prob_st



In [22]:
#payoff caso s responda à uma proposta de liquidação de j


def s_liq(teta_st, lst, ljt, t):
    
    st_liq = [Ls(t), V[t] * expec(teta_st, t, (t+1)) - Lj(t) ]

    policy_st_liq = np.argmax(st_liq)


    st_liq = st_liq[policy_st_liq]

    return (st_liq)




In [23]:
#testando para t = 15

Prob_s(0.5, 15)

s_liq(0.3, 1, 1, 15)

0.10000000000000009

13.0

In [24]:
#payoff caso s responda à uma proposta de reorganização de j

def s_reorg(teta_st, lst, ljt, t):
    
    st_reorg = [Ws[t], Ws[t]]

    policy_st_reorg = np.argmax(st_reorg)


    st_reorg = st_reorg[policy_st_reorg]
    
    return (st_reorg)



In [25]:
#testando para t = 15

s_reorg(0.6, 0.5, 0.5, 15)

#de fato a habilidade não muda s_reorg em t = 15 porque o próximo período é de liquidação
#nos outros períodos isso deve mudar

11

In [26]:
def st(teta_st, lst, ljt, t):
    
    #agregando todas as funções até agora
    
    st = (Prob_s(ljt,t)) * s_liq(teta_st, lst, ljt, t) + (1 - Prob_s(ljt,t)) * s_reorg(teta_st, lst, ljt, t)
    
    return (st)

In [27]:
#testando para t = 15

st(0.6, 0.5, 0.5, 15)

eval(st(0.6, 0.5, 0.5, 15))

11.5

ValueError: source code string cannot contain null bytes

In [34]:
#finalmente, a função valor de s no período t
#defini com subscrito _val porque o vetor Ws é argumento das funções serão passadas como argumento aqui

#havia uma confusão entre Ws função e Ws vetor

#percebi o erro com o link abaixo
#https://stackoverflow.com/questions/32446402/typeerror-function-object-is-not-subscriptable-in-python-3-4-3


def Ws_val(teta_st, lst, ljt, t):
    
    
    Wst = (λj) * St(teta_st, lst, ljt, t) + (1- λj) * st(teta_st, lst, ljt, t)
    
    return Wst

In [35]:
#testando para t = 15



Ws_val(0.5, 0.5, 0.5, 15)

#it works!!!!




13.2

Matriz tridimensional

In [ ]:
#usaremos uma matriz tridimensional para gravar as funções valor de cada período para as combinações das três variáveis


np.zeros((3,3,4))





In [ ]:
#retirada da habilidade de s
#é uma função da habilidade dele no período passado

#vetor com as habilidade realizadas em cada período
H_s = []
h_s0 = 0.1
H_s.append(h_s0)

#habilidade hoje é h_s, habilidade ontem é h_s1

h_s1 = H_s[len(H_s) -1]
def draw(x):
    h_s = np.random.uniform(x, 1)
    
    H_s.append(h_s) #atualiza com a habilidade realizada
    return h_s



### Solução do modelo simples

Como R[0] significa hoje, se o argmax de R for hoje, então a estratégia ótima é jogar Ps = L_j. Do contrário, vou supor que a oferta é menor que isso (será que tem diferença colocar Ps = 0 ou só Ps = L_j - 1?

In [ ]:
#código para computar em qual período s reorganiza a firma

def pbe(max_turnos, h_s0):
    
    #max_turnos vai ser calculada a partir de V, ρ e L. Por enquanto fica assim
    #max_turnos = 4 é o que estou testando
    #h_s0 é o nível de habilidade inicial do credor s
    t = 1
    
    
    
    #vetor que conterá as habilidades
    H_s = []
#     h_s0 = 0.1 #agora h_s0 é argumento da função
    H_s.append(h_s0)
    
    #vetor com as valores máximos de reorganização da firma a cada turno
    
    


    
    #vetor com os valores estimados de reorganização
    R = np.ones(max_turnos)
    R[0] = 0
    
    
    #faz um loop até que o valor máximo de reorganizar seja o de propor reorganizar em t
    while t < max_turnos and np.argmax(R) > 0:
        
        h_s = draw(h_s1)
    
        #matriz dos payoffs de reorganizar a firma
        R = np.empty(max_turnos-t + 1)


        # a última entrada de R é sempre o valor de liquidação
        R[len(R) -1] = L_s

        for i in range(t, max_turnos-t+1):
            R[i-t] = reorg(h_s, t, i)
        
        t = t + 1
    #retorna um vetor com três elementos: o período, a habilidade e o valor de Reorganização da firma
    return t, h_s, R[0]
    
    
    

### Estática Comparativa

Gráfico para mostrar como o valor de reorganização muda de acordo com a habilidade inicial do jogador s

In [ ]:
#gráfico para estática comparativa
grid_size = 25
H = np.empty(grid_size)

h_s0_vals = np.linspace(0.01, 0.99, grid_size)


R_vals = np.empty(grid_size)

for i in range(grid_size):
    R_vals[i] = pbe(4, h_s0_vals[i])[2]



fig, ax = plt.subplots()
ax.plot(R_vals, h_s0_vals, '-o', label='$q(w(i))$')
ax.set_title("Valor de Reorganização em relação à habilidade inicial")
ax.set_xlabel('Valor de Reorganização')
ax.set_ylabel('Habilidade Inicial')

plt.show()

#o valor de reorganização não é crescente na habilidade inicial.
#Acho que é por causa da aleatoriedade, então vou calcular a MÈDIA do valor de reorganização para vários draws com a mesma h_s0

### Estática Comparativa da média do Valor de Reorganização

In [ ]:
#gráfico para estática comparativa na MÉDIA

grid_size = 25
H = np.empty(grid_size)

h_s0_vals = np.linspace(0.01, 0.99, grid_size)


R_vals = np.empty(grid_size)

for i in range(grid_size):
       
    #calculando a média para os valores de retorno
    num_reps = 10000
    obs = np.empty(num_reps, dtype=np.float64)

    for rep in range(num_reps):
        obs[rep] = pbe(4,h_s0_vals[i])[2]
    


    R_vals[i] = obs.mean()



fig, ax = plt.subplots()
ax.plot(R_vals, h_s0_vals, '-o', label='$q(w(i))$')
ax.set_title("Valor de Reorganização MÈDIO em relação à habilidade inicial")
ax.set_xlabel('Valor de Reorganização')
ax.set_ylabel('Habilidade Inicial')

plt.show()

### Adicionando o valor de liquidação

O valor de liquidação dos jogadores no período t é

$L_s = min \{L - C_t, D_s \} $

e

$L_j = \min \big\{ L - C_t -  min \{L - C_t, D_s \} , D_j \big\}$


O custo de ir para a corte é $C_t = c_0D + c_1tD$, onde $c_o$ é o custo fixo e $c_1$ é variável

No artigo original, os autores usam $c_1 = 1,5\%$, que foi calibrado de acordo com a literatura. O parâmetro $c_0$ é estimado, aqui vamos supor inicialmente que é de 5%

### Calculando qual vai ser o período máximo de duração do jogo

O jogo termina quando o valor de reorganização cai e fica igual ao de liquidação:

$ \rho^T * V = L$

Vamos calcular esse T $\rho$, V e L. Ele será a expressão "max_turnos" no código de cada jogo.

Repare que o custo da corte $C_t$ é pago tanto na reorganização quanto na liquidação, então não precisamos considerá-lo:

O valor "excedente" da firma disponível para ser repartido entre os credores depende da habilidade de quem reorganizá-la
        $$ U_t (\theta_{k,t}) \equiv \theta_{k,t} V_{h,t} - C_t =\theta_{k,t} \rho^{t-1} V_{h,0} - C_t,     com   k \in \{s,j\} $$

In [ ]:


maximo_de_turnos(0.9, 100, 20)

In [ ]:
#checando os cálculos:

0.9 ** 16 * 100

L

0.9 ** 15 * 100

#o último turno é sempre de liquidação.



In [ ]:
#código atualizado para computar em qual período s reorganiza a firma 
#pbe2

def pbe2(ρ, Vmax, L, h_s0):
    
    #turnos do jogo
    t = 1
    max_turnos = maximo_de_turnos(ρ, Vmax, L)
    
    #vetor que conterá as habilidades
    #h_s0 é o nível de habilidade inicial do credor s
    H_s = []
#     h_s0 = 0.1 #agora h_s0 é argumento da função
    H_s.append(h_s0)
    
    #vetor com os valores estimados de reorganização
    R = np.ones(max_turnos)
    R[0] = 0
    
    
    #faz um loop até que o valor máximo de reorganizar seja o de propor reorganizar em t
    while t < max_turnos and np.argmax(R) > 0:
        
        h_s = draw(h_s1)
    
        #matriz dos payoffs de reorganizar a firma
        R = np.empty(max_turnos-t + 1)


        # a última entrada de R é sempre o valor de liquidação
        R[len(R) -1] = L_s

        for i in range(t, max_turnos-t+1):
            R[i-t] = reorg(h_s, t, i)
        
        t = t + 1
    #retorna um vetor com três elementos: o período, a habilidade e o valor de Reorganização da firma
    return t, h_s, R[0]

In [ ]:
maximo_de_turnos(0.9, 100, 50)

pbe2(0.9, 100, 50, 0.5)

# Próximos passos



* criar função de esperança baseada na função beta

* criar função de esperança de outras funções, ex: $\es{1}{\Ws{1}}$

* criar um vetor de W o valor final e ir preenchendo de trás pra frente

* pensar em como guardar as três dimensões referentes a teta, ls e lj para cada função valor

    * tem como fazer array tridimensional
    
* pensar em como colocar a função no período seguinte em função dos argumentos atuais








* passos finais
    * replicar os gráficos do artigo de referência
